In [1]:
from transformers import AutoTokenizer
from datasets import load_dataset
import tiktoken

# data

In [3]:
cc = load_dataset('HuggingFaceFW/fineweb-2', 'kor_Hang', cache_dir='../cache', split='train')
remove_columns = [i for i in cc.column_names if i!='text']
cc = cc.remove_columns(remove_columns)

README.md:   0%|          | 0.00/844k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/25 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/486 [00:00<?, ?it/s]

In [2]:
import os
from datasets import concatenate_datasets
from datasets import DatasetDict

In [4]:
cc_2 = load_dataset('HuggingFaceFW/fineweb-2', 'kor_Hang', cache_dir='../cache', split='test')
remove_columns = [i for i in cc_2.column_names if i!='text']
cc_2 = cc_2.remove_columns(remove_columns)

Resolving data files:   0%|          | 0/25 [00:00<?, ?it/s]

In [7]:
cc = concatenate_datasets([cc,cc_2])

In [3]:
total_data_path = '/home/work/g-earth-22/ocw/total/ko'

In [4]:
dataset_dict = {}
# dataset_dict['cc']=cc

In [5]:
from tqdm import tqdm

In [6]:
for name in tqdm(os.listdir(total_data_path)):
    path = os.path.join(total_data_path, name)
    datasets = []
    if os.path.isdir(path):
        for data_name in os.listdir(path):
            try:
                d = load_dataset('json', data_files=os.path.join(path, data_name), split='train')
            except:
                d = load_dataset('json', data_files=os.path.join(path, data_name))
            remove_columns = [i for i in d.column_names if i!='text']
            d = d.remove_columns(remove_columns)
            datasets.append(d)
    datasets = concatenate_datasets(datasets)
    dataset_dict[name]=datasets

 22% 2/9 [00:01<00:04,  1.41it/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/36 [00:00<?, ?it/s]

 78% 7/9 [00:11<00:02,  1.20s/it]

Loading dataset shards:   0%|          | 0/22 [00:00<?, ?it/s]

 89% 8/9 [00:14<00:01,  1.56s/it]

Loading dataset shards:   0%|          | 0/19 [00:00<?, ?it/s]

100% 9/9 [00:15<00:00,  1.70s/it]


In [7]:
_dataset_dict = dict()
for name, ds in tqdm(dataset_dict.items()):
    ds = ds.add_column('src',[name]*len(ds))
    _dataset_dict[name]=ds

100% 9/9 [00:57<00:00,  6.41s/it]


In [13]:
# cc = cc.add_column('src',['cc']*len(cc))
# _dataset_dict['cc']=cc

In [8]:
import json
import tiktoken
import base64

def load_tiktoken_tokenizer(file_path):
    from tiktoken.load import load_tiktoken_bpe
    with open(file_path, 'r') as f:
        tiktoken_json = json.loads(f.read())
    mergeable_ranks = {
      base64.b64decode(token): int(rank)
      for token, rank in (line.split() for line in tiktoken_json["mergeable_ranks"].splitlines() if line)
    }
    tokenizer = tiktoken.Encoding(
      name=file_path.split('/')[-1],
      pat_str=tiktoken_json['pat_str'],
      mergeable_ranks=mergeable_ranks,
      special_tokens=tiktoken_json['special_tokens']
    )
    return tokenizer

In [9]:
from itertools import chain
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, and if the total_length < block_size  we exclude this batch and return an empty dict.
    # We could add padding if the model supported it instead of this drop, you can customize this part to your needs.
    total_length = (total_length // block_size) * block_size        
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [10]:
def add_eos_token(text_list, eos_token='<|end_of_text|>'):
    output = []
    for i in text_list:
        output.append(i+eos_token)
    return output

In [11]:
# train test split
def train_test_split(ds, num=500):
    ds = ds.shuffle(seed=42)
    validation = ds.select(range(num))
    train = ds.select(range(num,len(ds)))
    return train, validation

In [12]:
# LOAD TOKENIZER
# ex) tokenizer = load_tiktoken_tokenizer('kt_tokenizer_v3/tiktoken/kt_v3.tiktoken')
TOKENIZER_MODEL_PATH='/home/work/g-earth-22/ocw/tokenizer/tiktoken/kt_v3.tiktoken'
tokenizer = load_tiktoken_tokenizer(f'{TOKENIZER_MODEL_PATH}')

tokenizer.encode('<|end_of_text|>', allowed_special='all')

[2]

In [13]:
block_size = 2048

# train-test dev

In [14]:
save_path = '../data/data_mix'

In [15]:
os.makedirs(save_path, exist_ok=True)

In [16]:
# for name, ds in tqdm(_dataset_dict.items()):
#     print(name)
#     train, validation = train_test_split(_dataset_dict[name])
#     ds_dict = DatasetDict({'train':train,'validation':validation})
#     ds_dict.save_to_disk(os.path.join(save_path, name), num_proc=64)
#     print('saved')

In [ ]:
for name, ds in tqdm(_dataset_dict.items()):
    print(name)
    train, validation = train_test_split(_dataset_dict[name])
    ds_dict = DatasetDict({'train':train,'validation':validation})
    ds_dict = ds_dict.map(
        lambda x: {"input_ids": tokenizer.encode_batch(add_eos_token(x),allowed_special='all')},
        input_columns='text',
        batched=True,
        num_proc=64)
    column_names = [i for i in ds_dict['train'].column_names if i not in ['input_ids']]
    _ds_dict = ds_dict.remove_columns(column_names)
    lm_datasets = _ds_dict.map(
            group_texts,
            batched=True,
            num_proc=64,
            desc=f"Grouping texts in chunks of {block_size}",
        )
    lm_datasets.save_to_disk(os.path.join(save_path, name), num_proc=64)

  0% 0/9 [00:00<?, ?it/s]

encyclo


Map (num_proc=64):   0%|          | 0/694015 [00:00<?, ? examples/s]

Map (num_proc=64):   0%|          | 0/500 [00:00<?, ? examples/s]

Grouping texts in chunks of 2048 (num_proc=64):   0%|          | 0/694015 [00:00<?, ? examples/s]

...

Saving the dataset (0/64 shards):   0%|          | 0/200220 [00:00<?, ? examples/s]

Saving the dataset (0/64 shards):   0%|          | 0/113 [00:00<?, ? examples/s]

 11% 1/9 [01:04<08:38, 64.81s/it]

cosmopedia


Map (num_proc=64):   0%|          | 0/2149493 [00:00<?, ? examples/s]